In [2]:
import json
import re

In [3]:
from google.colab import files
uploaded = files.upload()

Saving grafo_estrutura_dbt.ipynb to grafo_estrutura_dbt.ipynb
Saving load_file_to_bigquery.ipynb to load_file_to_bigquery.ipynb
Saving my_dbt_project.ipynb to my_dbt_project.ipynb
Saving read_data_from_bigquey.ipynb to read_data_from_bigquey.ipynb


In [4]:
# 2. Palavras-chave sensíveis para verificar
keywords = ['token', 'project_id', 'credentials', 'auth', 'key', 'password']

In [9]:
# Regex mais precisas para tokens
token_patterns = [
    r"AIza[0-9A-Za-z\-_]{35}",            # Google API Key
    r"ya29\.[0-9A-Za-z\-_]+",             # Google OAuth
    r"gh[pousr]_[0-9A-Za-z]{36,}",        # GitHub tokens
    r"xox[baprs]-[0-9A-Za-z-]+",          # Slack tokens
    r"(?i)(token|key|auth)[\"'\s:=]+[\"']?([A-Za-z0-9_\-]{20,})[\"']?"  # genérico com label
]

In [10]:
for filename in uploaded.keys():
    print(f"\n📁 Verificando arquivo: {filename}")

    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()

        notebook = json.loads(content)

        # Verifica célula a célula
        for i, cell in enumerate(notebook.get("cells", [])):
            if cell.get("cell_type") != "code":
                continue
            code_lines = cell.get("source", [])
            code = ''.join(code_lines)

            # Verificação por palavras-chave
            for keyword in keywords:
                for line in code_lines:
                    if re.search(rf"{keyword}\s*=\s*[\"']?.+?[\"']?", line, re.IGNORECASE):
                        print(f"⚠️  [Arquivo: {filename} | Célula: {i}] Palavra-chave suspeita: `{keyword}` → {line.strip()}")

            # Verificação por padrões de token
            for pattern in token_patterns:
                for line in code_lines:
                    matches = re.findall(pattern, line)
                    if matches:
                        for match in matches:
                            # Se a regex capturou grupos, pega só o token
                            if isinstance(match, tuple):
                                match = match[1]
                            print(f"🔑  [Arquivo: {filename} | Célula: {i}] Possível token encontrado: {match} → {line.strip()}")

        # project_id (fora de célula, no arquivo inteiro)
        project_ids = re.findall(r'project[-_]id["\']?\s*[:=]\s*["\']([\w-]+)', content, re.IGNORECASE)
        for project_id in project_ids:
            print(f"🆔 [Arquivo: {filename}] project_id detectado: {project_id}")

    except Exception as e:
        print(f"❌ Erro ao processar {filename}: {e}")


📁 Verificando arquivo: grafo_estrutura_dbt.ipynb

📁 Verificando arquivo: load_file_to_bigquery.ipynb
⚠️  [Arquivo: load_file_to_bigquery.ipynb | Célula: 3] Palavra-chave suspeita: `project_id` → PROJECT_ID = "meu-projeto-dbt"  # Substitua pelo seu project_id

📁 Verificando arquivo: my_dbt_project.ipynb
⚠️  [Arquivo: my_dbt_project.ipynb | Célula: 21] Palavra-chave suspeita: `project_id` → project_id = "meu-projeto-dbt"

📁 Verificando arquivo: read_data_from_bigquey.ipynb
⚠️  [Arquivo: read_data_from_bigquey.ipynb | Célula: 3] Palavra-chave suspeita: `project_id` → project_id = 'meu-projeto-dbt'  # substitua com seu ID do projeto
🆔 [Arquivo: read_data_from_bigquey.ipynb] project_id detectado: meu-projeto-dbt
